In [1]:
import collections
import csv
import math
import json
import os
import gzip
import pprint
import sys

import requests
#import pubchempy

In [2]:
# LINCS API variables
# http://api.lincscloud.org/
api_version = 'a2'
service = 'geneinfo'
api_url = 'http://api.lincscloud.org/{}/{}'.format(api_version, service)
api_key = '3d9912139ac9a1f5c388f28279488420'
block_size = 100 # max is 1000
print(api_url)

http://api.lincscloud.org/a2/geneinfo


In [10]:
def json_from_url(base, params):
    """Parse and return a json file obtained from the specified url."""
    response = requests.get(base, params=params)
    print response.url
    print response.text
    return response.json()

def query_links_api(query, verbose=False):
    """ """
    url_data = {'q': query, 'l': block_size, 'sk': 0, 'user_key': api_key, 'c': 'true', 's': '{"ds_index":0}', 'f':'{"pr_id":1}' }
    
    num_docs = json_from_url(api_url, url_data)['count']
    num_blocks = int(math.ceil(float(num_docs) / block_size))
    del url_data['c']
    if verbose:
        print '{} results: splitting query into {} chunks of {}.'.format(num_docs, num_blocks, block_size)
    
    results = list()
    for i in range(num_blocks):
        if verbose:
            print 'Chunk {}/{}'.format(i + 1, num_blocks)
        url_data['sk'] =  i * block_size
        results += json_from_url(api_url, url_data)
    
    return results

#for i in range(num_blocks):
#        if verbose:
#            print 'Chunk {}/{}'.format(i + 1, num_blocks)
#        url_data['sk'] =  i * block_size
#        results += json_from_url(api_url, url_data)

In [11]:
query = '{"is_lm":true}'
compounds = query_links_api(query)

http://api.lincscloud.org/a2/geneinfo?q=%7B%22is_lm%22%3Atrue%7D&c=true&f=%7B%22pr_id%22%3A1%7D&user_key=3d9912139ac9a1f5c388f28279488420&l=100&sk=0&s=%7B%22ds_index%22%3A0%7D
{
  "count": 1171
}
http://api.lincscloud.org/a2/geneinfo?q=%7B%22is_lm%22%3Atrue%7D&f=%7B%22pr_id%22%3A1%7D&user_key=3d9912139ac9a1f5c388f28279488420&l=100&sk=0&s=%7B%22ds_index%22%3A0%7D
[
  {
    "_id": "51438660dfe7719b8846d2b4",
    "pr_id": "200814_at"
  },
  {
    "_id": "51438660dfe7719b8846d2b5",
    "pr_id": "222103_at"
  },
  {
    "_id": "51438660dfe7719b8846d2b6",
    "pr_id": "201453_x_at"
  },
  {
    "_id": "51438660dfe7719b8846d2b7",
    "pr_id": "204131_s_at"
  },
  {
    "_id": "51438660dfe7719b8846d2b8",
    "pr_id": "200059_s_at"
  },
  {
    "_id": "51438660dfe7719b8846d2b9",
    "pr_id": "205067_at"
  },
  {
    "_id": "51438660dfe7719b8846d2ba",
    "pr_id": "213702_x_at"
  },
  {
    "_id": "51438660dfe7719b8846d2bb",
    "pr_id": "214435_x_at"
  },
  {
    "_id": "51438660dfe7719b8846d2b

In [15]:
compound_list = []
for i in range(len(compounds)):
    compound_list.append(compounds[i]['pr_id'])
    

In [18]:
with open('/Users/leobrueggeman/GitHub/LINCS/islm_probe_list.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    for val in compound_list:
        writer.writerow([val])    
 

In [2]:
directory = '/home/dhimmels/Documents/serg/myelin/lincs/'

In [8]:
#path = os.path.join(directory, 'lincs_small_molecules.json.gz')
with gzip.open('/Users/leobrueggeman/GitHub/LINCS/islm_probe_list.json', 'w') as write_file:
    json.dump(compounds, write_file)

## Start here to read lincs catalog from json rather than REST api
query pubchem for parent compounds

In [ ]:
directory = '/home/dhimmels/Documents/serg/myelin/lincs/'
path = os.path.join(directory, 'lincs_small_molecules.json.gz')
with gzip.open(path) as read_file:
    compounds = json.load(read_file)

In [ ]:
# Set missing pubchem_cid records to None
for compound in compounds:
    pubchem_cid = compound.get('pubchem_cid')
    try:
        pubchem_cid = int(pubchem_cid)
    except (ValueError, TypeError) as e:
        compound['pubchem_cid'] = None
        continue
    if pubchem_cid <= 0:
        compound['pubchem_cid'] = None

In [ ]:
def get_pubchem_parent(cid, orphans_as_self=True):
    """
    From a pubchem_cid, retreive the parent compound's cid.
    If function is unsuccesful in retrieving a single parent,
    `orphans_as_self = True` returns `cid` rather than None.
    
    According to pubmed:
    
    > A parent is conceptually the "important" part of the molecule
    > when the molecule has more than one covalent component.
    > Specifically, a parent component must have at least one carbon
    > and contain at least 70% of the heavy (non-hydrogen) atoms of
    > all the unique covalent units (ignoring stoichiometry).
    > Note that this is a very empirical definition and is subject to change.

    A parallel query can be executed using the REST PUG API:
    http://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/11477084/cids/XML?cids_type=parent
    """
    assert cid
    
    try:
        parent_cids = pubchempy.get_cids(identifier=cid, namespace='cid', domain='compound', cids_type='parent')
    except pubchempy.BadRequestError as e:
        print 'Error getting parent of {}. {}'.format(cid, e)
        return cid if orphans_as_self else None
    try:
        parent_cid, = parent_cids
        return parent_cid
    except ValueError:
        print 'Error getting parent of {}. Parents retreived: {}'.format(cid, parent_cids)
    return cid if orphans_as_self else None

In [ ]:
for i, compound in enumerate(compounds):
    if i % 1000 == 0:
        print '{} / {} compounds complete'.format(i, len(compounds))
    if 'parent_cid' in compound:
        #print 'parent cached for {}'.format(compound['pert_iname'])
        continue
    pubchem_cid = compound.get('pubchem_cid')
    if not pubchem_cid:
        print 'No pubchem_cid record for {}'.format(compound['pert_iname'])
        compound['parent_cid'] = None
        continue
    compound['parent_cid'] = get_pubchem_parent(pubchem_cid)
    sys.stdout.flush()

In [ ]:
# http://api.lincscloud.org/a2/docs/pertinfo
fields = ['pert_id', 'pert_iname', 'pubchem_cid', 'parent_cid', 'molecular_formula',
          'molecular_wt', 'pert_vendor', 'num_gold', 'num_inst', 'num_sig', 
          'canonical_smiles', 'inchi_key', 'inchi_string']

path = os.path.join(directory, 'lincs_small_molecules.txt')
with open(path, 'w') as write_file:
    writer = csv.DictWriter(write_file, delimiter='\t', fieldnames=fields, extrasaction='ignore')
    writer.writeheader()
    writer.writerows(compounds)

In [3]:
# read LINCS compounds from file
path = os.path.join(directory, 'lincs_small_molecules.txt')
with open(path) as read_file:
    reader = csv.DictReader(read_file, delimiter='\t')
    compounds = list(reader)

### Read the myelin screen results

In [35]:
screen_directory = '/home/dhimmels/Documents/serg/myelin/screen'
path = os.path.join(screen_directory, 'compounds', 'screens-pubchem.txt')
with open(path) as read_file:
    reader = csv.DictReader(read_file, delimiter='\t')
    screen_fields = reader.fieldnames
    screen = list(reader)
screen_fields

['screen',
 'pubchem_cid',
 'parent_cid',
 'screen_name',
 'screen_catalog_id',
 'mean_PDGFR',
 'mean_MBP',
 'SEM_PDGFR',
 'SEM_MBP',
 'canonical_smiles_SEA']

In [36]:
len(screen)

1021

In [37]:
lincs_compound_cids  = {str(c['pubchem_cid']) for c in compounds if c.get('pubchem_cid')}
screen_compound_cids = {str(c['pubchem_cid']) for c in screen    if c.get('pubchem_cid')}

lincs_parent_cids  = {str(c['parent_cid']) for c in compounds if c.get('parent_cid')}
screen_parent_cids = {str(c['parent_cid']) for c in screen    if c.get('parent_cid')}

print 'Myelin Screen: {} pubchem compounds, {} pubchem parent compounds'.format(len(screen_compound_cids), len(screen_parent_cids))
print 'LINCS: {} pubchem compounds, {} pubchem parent compounds'.format(len(lincs_compound_cids), len(lincs_parent_cids))
print '{} compounds match, {} parents match'.format(
    len(screen_compound_cids & lincs_compound_cids),
    len(screen_parent_cids & lincs_parent_cids))

Myelin Screen: 971 pubchem compounds, 962 pubchem parent compounds
LINCS: 20136 pubchem compounds, 20120 pubchem parent compounds
446 compounds match, 488 parents match


In [38]:
pubchem_cid_to_lincs = dict()
parent_cid_to_lincs = dict()

for compound in compounds:
    pubchem_cid = compound['pubchem_cid']
    parent_cid = compound['parent_cid']
    pubchem_cid_to_lincs.setdefault(pubchem_cid, list()).append(compound)
    parent_cid_to_lincs.setdefault(parent_cid, list()).append(compound)


print collections.Counter(map(len, pubchem_cid_to_lincs.values()))
print collections.Counter(map(len, parent_cid_to_lincs.values()))

Counter({1: 19947, 2: 188, 3: 1, 87: 1})
Counter({1: 19912, 2: 206, 7: 1, 3: 1, 79: 1})


In [39]:
for compound in screen:
    if 'lincs_pert_ids' in compound:
        del compound['lincs_pert_ids']
    
    pubchem_cid = compound['pubchem_cid']
    parent_cid = compound['parent_cid']
    
    for screen_level, screen_cid in ('compound', pubchem_cid), ('parent', parent_cid):
        if compound.get('lincs_pert_ids'):
            continue
        for lincs_level, cid_to_lincs in ('compound', pubchem_cid_to_lincs), ('parent', parent_cid_to_lincs):
            if compound.get('lincs_pert_ids'):
                continue
            lincs_matches = cid_to_lincs.get(screen_cid, list())
            matched_pert_ids = list({match['pert_id'] for match in lincs_matches})
            compound['lincs_pert_ids'] = matched_pert_ids
            if matched_pert_ids:
                compound['lincs_match_method'] = 'screen-{}_lincs-{}'.format(screen_level, lincs_level)
            if len(matched_pert_ids) > 1:
                print 'Screen {} {} matches multiple LINCS {}s: {}'.format(screen_level, screen_cid, lincs_level, matched_pert_ids)

collections.Counter(bool(compound['lincs_pert_ids']) for compound in screen)

Screen compound 2756 matches multiple LINCS compounds: ['BRD-K34157611', 'BRD-K18618618']
Screen compound 3476 matches multiple LINCS compounds: ['BRD-K34776109', 'BRD-K42693031']
Screen compound 4201 matches multiple LINCS compounds: ['BRD-K14888893', 'BRD-K06902185']
Screen compound 4497 matches multiple LINCS compounds: ['BRD-A24891640', 'BRD-A58048407']
Screen compound 4499 matches multiple LINCS compounds: ['BRD-A96107863', 'BRD-A84465106']
Screen compound 5447130 matches multiple LINCS compounds: ['BRD-K53815609', 'BRD-K79092138']
Screen compound 445154 matches multiple LINCS compounds: ['BRD-K25591257', 'BRD-K80738081']
Screen compound 5282230 matches multiple LINCS compounds: ['BRD-K17849083', 'BRD-K19533706']
Screen compound 444795 matches multiple LINCS compounds: ['BRD-K06926592', 'BRD-K71879491']
Screen parent 3006531 matches multiple LINCS compounds: ['BRD-K18787491', 'BRD-K46419649']
Screen compound 5494449 matches multiple LINCS compounds: ['BRD-K59369769', 'BRD-K8794736

Counter({True: 522, False: 499})

## Compute Pubchem 2D similarities
Used [online query](https://pubchem.ncbi.nlm.nih.gov/score_matrix/score_matrix.cgi). From [help page](https://pubchem.ncbi.nlm.nih.gov/score_matrix/score_matrix-help.html):

> 2D Similarity: Substructure key-based 2D Tanimoto similarity. Scores are in the range [0 .. 100].

In [40]:
screen_cids = sorted(map(int, screen_compound_cids & screen_parent_cids))
path = os.path.join(screen_directory, 'compounds', 'screens-pubchem-cids.txt')
with open(path, 'w') as write_file:
    write_file.write('\n'.join(map(str, screen_cids)))

lincs_cids = sorted(map(int, lincs_compound_cids & lincs_parent_cids))
path = os.path.join(screen_directory, 'compounds', 'lincs-pubchem-cids.txt')
with open(path, 'w') as write_file:
    write_file.write(','.join(map(str, lincs_cids)))

In [10]:
import gzip

path = os.path.join(screen_directory, 'compounds', 'screen-lincs-similarities', '2d-similarity.tab.gz')
with gzip.open(path) as read_file:
    # fieldnames=['cid_screen', 'cid_lincs', 'score']
    reader = csv.reader(read_file, delimiter='\t')
    similarities = list()
    for similarity in reader:
        score = int(similarity[2])
        if score > 95:
            similarity[2] = score
            similarities.append(similarity)

len(similarities)

1118

In [41]:
similarity_threshold = 100

similarity_mapper = dict()
for cid_screen, cid_lincs, score in similarities:
    if score < similarity_threshold:
        continue
    similarity_mapper.setdefault(cid_screen, list()).append(cid_lincs)

len(similarity_mapper)

502

In [42]:
def get_lincs_perts(cid):
    lincs_matches = pubchem_cid_to_lincs.get(cid, list())
    if lincs_matches:
        return list({match['pert_id'] for match in lincs_matches})
    lincs_matches = parent_cid_to_lincs.get(cid, list())
    return list({match['pert_id'] for match in lincs_matches})

def combine_lincs_perts(cids):
    all_perts = set()
    for cid in cids:
        perts = get_lincs_perts(cid)
        all_perts |= set(perts)
    return list(all_perts)

In [43]:
for compound in screen:
    lincs_pert_ids = compound['lincs_pert_ids']
    if lincs_pert_ids:
        continue
        
    pubchem_cid = compound['pubchem_cid']
    if pubchem_cid in similarity_mapper:
        lincs_cids = similarity_mapper[pubchem_cid]
        compound['lincs_pert_ids'] = combine_lincs_perts(lincs_cids)
        compound['lincs_match_method'] = 'compound_similarity'
        continue

    parent_cid = compound['parent_cid']
    if parent_cid in similarity_mapper:
        lincs_cids = similarity_mapper[parent_cid]
        compound['lincs_pert_ids'] = combine_lincs_perts(lincs_cids)
        compound['lincs_match_method'] = 'parent_similarity'
        continue

collections.Counter(bool(compound['lincs_pert_ids']) for compound in screen)

Counter({True: 596, False: 425})

In [44]:
screen_directory = '/home/dhimmels/Documents/serg/myelin/screen'
path = os.path.join(screen_directory, 'compounds', 'screens-pubchem.txt')

with open(path, 'w') as write_file:
    if 'lincs_pert_ids' not in screen_fields:
        screen_fields += ['lincs_pert_ids']
    if 'lincs_match_method' not in screen_fields:
        screen_fields += ['lincs_match_method']
    writer = csv.DictWriter(write_file, delimiter='\t', fieldnames = screen_fields)
    writer.writeheader()
    writer.writerows(screen)
screen_fields

['screen',
 'pubchem_cid',
 'parent_cid',
 'screen_name',
 'screen_catalog_id',
 'mean_PDGFR',
 'mean_MBP',
 'SEM_PDGFR',
 'SEM_MBP',
 'canonical_smiles_SEA',
 'lincs_pert_ids',
 'lincs_match_method']

## Downloading LINCS expression

In [ ]:
len

In [23]:
doesn = 'http://api.lincscloud.org/a2/geneinfo?q={%22is_bing%22:true}&s={%22ds_index%22:1}&f={%22ds_index%22:1,%22pr_id%22:1,%22pr_gene_symbol%22:1,%22pr_gene_id%22:1}&sk=0&l=100&userkey=3d9912139ac9a1f5c388f28279488420'

In [20]:
works = 'http://api.lincscloud.org/a2/geneinfo?q={%22is_bing%22:true}&s={%22ds_index%22:1}&f={%22pr_id%22:1,%22ds_index%22:1,%22pr_gene_symbol%22:1,%22pr_gene_id%22:1}&sk=0&l=100&user_key=3d9912139ac9a1f5c388f28279488420'

In [24]:
print(len(works))
len(doesn)

211


210